# 토치텍스트 튜토리얼(한글)

## 1. 형태소 분석기 Mecab 설치

In [1]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 58.1 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-01-05 06:12:37--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::3403:4be7, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=hnkNlVPCnN7ImVqNGe0a2%2FDO314%3D&Expires=1641364217

## 2. 훈련 데이터와 테스트 데이터로 다운로드 하기
네이버 영화 리뷰 데이터를 다운로드합니다. 훈련 데이터는 ratings_train.txt, 테스트 데이터는 ratings_test.txt에 입니다.

In [3]:
import urllib.request
import pandas as pd
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f90ed412a50>)

In [5]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [6]:
train_df.head()

id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1

In [7]:
test_df.head()

id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0

In [9]:
print("훈련 데이터 샘플의 개수 : {}" .format(len(train_df)))
print("테스트 데이터 샘플의 개수 : {}" .format(len(test_df)))

훈련 데이터 샘플의 개수 : 150000
테스트 데이터 샘플의 개수 : 50000


## 3. 필드 정의하기 (torchtext.data)

In [13]:
from torchtext.legacy import data # torchtext.data 임포트
from konlpy.tag import Mecab

In [14]:
tokenizer = Mecab()

In [23]:
ID = data.Field(sequential= False,
                use_vocab = False)
TEXT = data.Field(sequential = True,
                  use_vocab = True,
                  tokenize = tokenizer.morphs,
                  lower = True,
                  batch_first = True,
                  fix_length=20)
LABEL = data.Field(sequential = False,
                   use_vocab = False,
                   is_target=True)

## 4. 데이터셋 만들기

In [24]:
from torchtext.legacy.data import TabularDataset

In [27]:
train_data, test_data = TabularDataset.splits(
    path = '.', train = 'ratings_train.txt', test = 'ratings_test.txt', format='tsv',
    fields = [('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True
)

In [28]:
print("훈련 샘플의 개수 : {}" .format(len(train_data)))
print("테스트 샘플의 개수 : {}" .format(len(test_data)))

훈련 샘플의 개수 : 150000
테스트 샘플의 개수 : 50000


In [33]:
print(vars(train_data[0]))
print(vars(train_data[1]))

{'id': '9976970', 'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}
{'id': '3819312', 'text': ['흠', '.', '..', '포스터', '보고', '초딩', '영화', '줄', '.', '...', '오버', '연기', '조차', '가볍', '지', '않', '구나'], 'label': '1'}


## 5. 단어 집합(vacabulary) 만들기

In [35]:
#  .build_vocab() 도구를 사용하면 단어 집합을 생성
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)

In [36]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [38]:
print(TEXT.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7f8fca07f150>>, {'<unk>': 0, '<pad>': 1, '.': 2, '이': 3, '는': 4, '영화': 5, '다': 6, '고': 7, '하': 8, '도': 9, '의': 10, '가': 11, '은': 12, '에': 13, '을': 14, '보': 15, '한': 16, '..': 17, '게': 18, ',': 19, '들': 20, '!': 21, '지': 22, '를': 23, '있': 24, '없': 25, '?': 26, '좋': 27, '나': 28, '었': 29, '만': 30, '는데': 31, '너무': 32, '봤': 33, '적': 34, '안': 35, '정말': 36, '로': 37, '음': 38, '으로': 39, '것': 40, '아': 41, '네요': 42, '재밌': 43, '어': 44, '점': 45, '같': 46, '지만': 47, '진짜': 48, '했': 49, '에서': 50, '기': 51, '네': 52, '않': 53, '거': 54, '았': 55, '수': 56, '되': 57, '면': 58, 'ㅋㅋ': 59, '과': 60, '말': 61, '연기': 62, '인': 63, '잘': 64, '최고': 65, '주': 66, '내': 67, '~': 68, '평점': 69, '이런': 70, '어요': 71, '던': 72, '와': 73, '할': 74, '왜': 75, '1': 76, '겠': 77, '해': 78, '스토리': 79, '습니다': 80, 'ㅋㅋㅋ': 81, '...': 82, '드라마': 83, '생각': 84, '아니': 85, '더': 86, '그': 87, '싶': 88, '사람': 89, '듯': 90, '감동': 91, '때': 92, '함': 93, '배우': 94, '본

## 6. 토치텍스트의 데이터로더 만들기
데이터로더는 데이터셋에서 미니 배치만큼 데이터를 로드하게 만들어주는 역할을 합니다. 토치텍스트에서는 Iterator를 사용하여 데이터로더를 만듭니다.

In [39]:
from torchtext.legacy.data import Iterator

In [40]:
train_loader = Iterator(dataset=train_data, batch_size = 5)
test_loader = Iterator(dataset=test_data, batch_size = 5)

In [45]:
print("훈련 데이터의 미니 배치 수 : {}" .format(len(train_loader)))
print("테스트 데이터의 미니 배치 수 : {}" .format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 30000
테스트 데이터의 미니 배치 수 : 10000


In [46]:
batch = next(iter(train_loader))
print(batch.text)

tensor([[  36,  110,   18,   95,    5,   68,  400,    9,   27,    7,   68,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [5810,   22,    9,   53,   18, 3331,   51,  210,   32, 1039,   32,  837,
          118,    1,    1,    1,    1,    1,    1,    1],
        [ 398,   34,   39,  102,    9,    0,    7,   97,   85,  116,   80,  167,
           66,    7,   15,   51, 1427,    0,  183,   88],
        [ 239,    5,   34,   63, 4572,   50,  396,   12,   64, 2130,   38,  419,
          583,  371,   67,   11, 1747,  182,  536, 3296],
        [ 753,    9,   27,   41,   49,    7,  218,    9,   32,   64,   33,   71,
            1,    1,    1,    1,    1,    1,    1,    1]])
